<a href="https://colab.research.google.com/github/mpsdecamargo/ml-data-science-portfolio/blob/main/bert-deep-learning-project/Load_and_Predict.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## INTRODUCTION

Notebook content: Loading of models and prediction of sample corpus. Sample of final function to save output into JSON format to be sent to the front end of the application and to the user.

In [1]:
!pip install sentencepiece transformers sentence_transformers datasets


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 5.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 13.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 4.9 MB/s eta 0:00:00
  Created wheel for sentence_transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125923 sha256=3fda6e7c9e1bf331d3ad78f0abc5a8302f98bf4dea4d946423e8223ef2a9733b
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence_transformers


In [2]:
!pip install accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 1.7 MB/s eta 0:00:00


In [3]:
import sentencepiece
import pandas as pd
import pandas as pd
import numpy as np
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
import json
import time
import operator
import re
from sentence_transformers import SentenceTransformer, util
import pickle
import torch
from datasets import Dataset
from scipy.special import expit
import io
from contextlib import redirect_stdout

In [8]:
lang_models = {"BERTPT":"neuralmind/bert-base-portuguese-cased","BERTPTL":"neuralmind/bert-large-portuguese-cased","MBERT":"bert-base-multilingual-cased", "ELECTRA":"dlb/electra-base-portuguese-uncased-brwac","ROBERTA": "rdenadai/BR_BERTo","XLMR":"xlm-roberta-base","DISTILBERT": "distilbert-base-multilingual-cased","ALBERT":"josu/albert-pt-br","DEBERTA":"microsoft/mdeberta-v3-base"}


In [5]:
from google.colab import drive
drive.mount('gdrive')

Mounted at gdrive


In [9]:
# Defining of the input data to be analysed
# The first 5 samples are true (label = 0)
# The last 5 are cases of disinformation (label = 1)

prediction_corpus = [

'Cientistas dizem haver "evidências convincentes" de que o mercado de frutos do mar e vida selvagem de Huanan, na cidade chinesa de Wuhan, foi o epicentro do surto de covid-19. Isso sugere que houve duas variantes introduzidas em humanos em novembro ou início de dezembro de 2019. Os pesquisadores dizem que há evidências de que o Sars-Cov-2 estava presente em mamíferos vivos que foram vendidos no mercado de Huanan no final de 2019. Cientistas dizem que o coronavírus foi transmitido para pessoas que estavam trabalhando ou fazendo compras lá em duas "infecções por transbordamento" separadas — em que um humano contraiu o vírus de um animal.',

'No decorrer da pandemia de covid-19, descobrimos quem pode estar em maior risco de contrair o vírus e desenvolver os casos mais graves da doença. Idade avançada, obesidade e várias comorbidades, como diabetes e hipertensão, são alguns desses fatores de risco. O alto índice de massa corporal (IMC) também parece estar associado ao aumento das chances de contrair o vírus, em primeiro lugar. Mas e os fatores que podem tornar alguém menos propenso a contrair covid? Curiosamente, pesquisas apontam que ter alergias pode reduzir o risco. As alergias são muito comuns. Pelo menos 400 milhões de pessoas em todo o mundo são afetadas por alergias ao pólen, conhecida em inglês como hay fever. Cerca de 300 milhões de pessoas sofrem de asma alérgica (induzida pela inalação de alérgenos), enquanto as alergias alimentares afetam cerca de 250 milhões. Muitas pessoas também são alérgicas a certos medicamentos. As reações alérgicas podem variar de leves (talvez alguma vermelhidão e inchaço na pele) a graves (choque anafilático, que pode causar a morte)',

'Eficácia da vacina contra Covid-19 em gestantes é confirmada; menos de 10% apresentaram reações adversas. A pesquisa avaliou dados de mais de 85 mil partos realizados no Canadá e nos EUA no ano de 2021.',

'As subvariantes BA.4 e BA.5 do novo coronavírus conseguem escapar dos anticorpos de pessoas que tiveram infecção anterior por Covid-19 e aquelas que já receberam a dose de reforço da vacina, de acordo com novos dados de pesquisadores da Escola de Medicina de Harvard. No entanto, a vacinação contra a Covid-19 ainda fornece proteção substancial contra formas graves da doença, e os fabricantes de imunizantes estão trabalhando na atualização de vacinas contra as variantes. Os níveis de anticorpos neutralizantes que uma infecção anterior ou vacinação provocam são várias menores contra as subvariantes BA.4 e BA.5, da Ômicron, em comparação com a cepa original do novo coronavírus, de acordo com a nova pesquisa publicada no New England Journal of Medicine na quarta-feira (22).',

'O presidente americano, Joe Biden, disse neste domingo (18) que a pandemia de Covid-19 acabou nos Estados Unidos. "A pandemia terminou", disse em entrevista à TV. "Ainda temos um problema com a Covid. Ainda estamos trabalhando muito nisso... Mas a pandemia acabou. Se prestarem atenção, ninguém usa máscaras. Todo mundo parece estar em boa forma. E, por isso, acredito que está mudando", afirmou durante o programa "60 Minutes", da rede CBS, durante a feira de automóveis de Detroit, evento que não acontecia havia três anos.',

'EXPOSIÇÃO MICROSCÓPICA DAS VACINAS Em SP, um telão exibe o que um microscópio revela estar presente em uma única gota das vacinas Pfizer, AstraZeneca, Johnson e CoronaVac. Pode ser ver os nano dispositivos, sensores e muitas bactérias',

'URGENTE-HARVARD COMPROVA EFICÁCIA DA HIDROXICLOROQUINA Em estudos randomizados, controlados por placebo, duplo cego, pessoas designadas para uso de hidroxicloroquina tiveram 28% menos risco de COvid 19, afirmou Dr, Miguel Herman, professor de Havard. E AGORA?',

'Desde o lançamento das vacinas de mRNA COVID-19, muitos pesquisadores especularam sobre a possibilidade de transmissão de partículas vacinais do vacinado para o não vacinado, mais comumente conhecido como “derramamento de vacina”. Um recente estudo de pré-impressão atraiu a atenção da mídia, pois os resultados do estudo podem ser interpretados como sugestivos de transmissão da vacina por disseminação. Desde o lançamento das vacinas de mRNA COVID-19, muitos pesquisadores especularam sobre a possibilidade de transmissão de partículas vacinais do vacinado para o não vacinado, mais comumente conhecido como “derramamento de vacina”. Embora essas especulações sejam rotineiramente verificadas, histórias anedóticas de pessoas não vacinadas que foram infectadas com COVID-19 ou experimentaram sintomas estranhos após o contato com amigos ou familiares que foram vacinados persistiram. Vários médicos também especularam sobre uma possível transmissão à medida que surgem novas descobertas em apoio a essas ideias. Um recente estudo de pré-impressão atraiu a atenção da mídia, pois os resultados do estudo podem ser interpretados como sugestivos de transmissão da vacina por disseminação.',

'Um novo estudo prospectivo que inclui mais de 223.000 participantes do sul do Brasil descobriu que a ivermectina reduz a mortalidade por covid-19 em 92% . O uso de ivermectina foi associado a reduções não apenas na mortalidade por COVID-19, mas também nas hospitalizações e nas taxas de infecção. Como profilático, a ivermectina mostrou-se mais eficaz de maneira dose-dependente. À medida que a dose de ivermectina aumentava, os pacientes tinham maior probabilidade de se recuperar da covid-19. Evidências preliminares também mostram que os pacientes se recuperam mais rapidamente com ivermectina .',

'O câncer começa quando as alterações genéticas interferem na replicação normal e na reposição das células do corpo. As células começam a crescer descontroladamente e podem formar um tumor. É a segunda principal causa de morte nos Estados Unidos. Infelizmente, parece que a doença pode estar aumentando graças às injeções experimentais de Covid-19. Porque os dados oficiais do governo dos EUA confirmam que o risco de desenvolver câncer após a vacinação contra o Covid-19 aumenta em chocantes 143.233%'

]

In [10]:
# Loading the tokenizer and creation of tokenize function.
# The tokenizer transforms the text data into tokens, which are essentially different numbers,
# which are then used in training.

def startTokenizer(model_type):
  load = time.time()
  tokenizer = AutoTokenizer.from_pretrained(lang_models[model_type])
  end = time.time()
  tokenizationLoad = end - load
  print("Loading Tokenization: ", tokenizationLoad)
  return tokenizer

def tokenize(text, tokenizer):
    tokenized_input = tokenizer(
        text,
        truncation=True,
        max_length=512,
        padding='max_length',
        return_tensors='np',
        return_attention_mask=True,
        return_token_type_ids=True,
    )
    return {
        'input_ids': tokenized_input['input_ids'][0],
        'attention_mask': tokenized_input['attention_mask'][0],
        'token_type_ids': tokenized_input['token_type_ids'][0],
    }

In [11]:
# Defining the repositories of the pretrained language models to be used

model_paths = {
               "BERTPT":"gdrive/My Drive/Modelos/BERTPT/BERTPT_Model_2024-01-02_23-23",
               "BERTPTL":"gdrive/My Drive/Modelos/BERTPTL/BERTPTL_Model_2024-01-03_16-05",
               "MBERT":"gdrive/My Drive/Modelos/MBERT/MBERT_Model_2024-01-03_01-17",
               "ELECTRA":"gdrive/My Drive/Modelos/ELECTRA/ELECTRA_Model_2024-01-02_22-21",
               "ROBERTA": "gdrive/MyDrive/Modelos/ROBERTA/ROBERTA_Model_2024-01-06_21-14",
               "XLMR":"gdrive/My Drive/Modelos/XLMR/XLMR_Model_2024-01-05_16-19",
               "DISTILBERT": "gdrive/My Drive/Modelos/DISTILBERT/DISTILBERT_Model_2024-01-02_21-13",
               "ALBERT":"gdrive/My Drive/Modelos/ALBERT/ALBERT_Model_2024-01-02_17-54",
               "DEBERTA":"gdrive/My Drive/Modelos/DEBERTA/DEBERTA_Model_2024-01-03_10-24",
               "BERTPT_ML": "gdrive/My Drive/Modelos/MultiLabel/BERTPT/BERTPT_MultiLabelModel_2024-01-03_20-24",
               "DEBERTA_ML": "gdrive/My Drive/Modelos/MultiLabel/DEBERTA/DEBERTA_MultiLabelModel_2024-01-03_20-43"
               }

In [12]:
# Function loaded_model_predict loads the tokenizer and the model, tokenizes the dataset,
# sets configuration of the parameters of the Trainer and uses the loaded model to predict based on input data.
# If the multilabel parameter is defined to true, the configuration is changed and, also, the function returns the prediction for each theme instead.

def loaded_model_predict(model_type,dataset=prediction_corpus, multilabel=False):
  start = time.time()
  loaded_tokenizer = AutoTokenizer.from_pretrained(model_paths[model_type])
  loaded_model = AutoModelForSequenceClassification.from_pretrained(model_paths[model_type])
  if multilabel == True:
    loaded_model.config.problem_type = "multi_label_classification"
  tokenized_prediction_corpus = [tokenize(text, loaded_tokenizer) for text in dataset]
  tokenized_dataset = Dataset.from_dict({"input_ids": [item['input_ids'] for item in tokenized_prediction_corpus],
                                           "attention_mask": [item['attention_mask'] for item in tokenized_prediction_corpus],
                                           "token_type_ids": [item['token_type_ids'] for item in tokenized_prediction_corpus]})
  training_args = TrainingArguments(
        output_dir=f"./test_trainer/{model_type}",
        do_train = False,
        do_predict = True,
        )

  trainer = Trainer(
      model=loaded_model,
      args=training_args,
  )

  results = trainer.predict(tokenized_dataset)
  if multilabel:
    predicted_labels = np.round(expit(np.array(results.predictions)))
  else:
    predicted_labels = np.argmax(results.predictions, axis=-1)
  end = time.time()
  pred_time = end - start
  print(predicted_labels)
  print(f"Prediction Time: {pred_time}")
  return predicted_labels

# CORPUS PREDICTION

In [ ]:
loaded_model_predict("BERTPT")

[0 0 1 0 0 1 1 1 1 1]
Prediction Time: 10.95037293434143


array([0, 0, 1, 0, 0, 1, 1, 1, 1, 1])

In [ ]:
loaded_model_predict("ALBERT")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


[0 1 1 0 0 1 1 0 1 1]
Prediction Time: 2.3080694675445557


array([0, 1, 1, 0, 0, 1, 1, 0, 1, 1])

In [ ]:
loaded_model_predict("ELECTRA")

[1 0 1 0 0 1 1 0 1 1]
Prediction Time: 17.68340516090393


array([1, 0, 1, 0, 0, 1, 1, 0, 1, 1])

In [ ]:
loaded_model_predict("MBERT")

[0 0 1 0 0 1 1 1 1 1]
Prediction Time: 13.806703567504883


In [ ]:
loaded_model_predict("DISTILBERT")

[0 0 1 0 0 1 1 0 1 1]
Prediction Time: 17.033589601516724


array([0, 0, 1, 0, 0, 1, 1, 0, 1, 1])

In [ ]:
loaded_model_predict("DEBERTA")

/usr/local/lib/python3.10/dist-packages/transformers/convert_slow_tokenizer.py:473: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


[1 0 1 0 0 1 1 1 1 1]
Prediction Time: 25.240070819854736


array([1, 0, 1, 0, 0, 1, 1, 1, 1, 1])

In [ ]:
loaded_model_predict("ROBERTA")

[1 1 1 0 0 1 1 0 1 1]
Prediction Time: 19.10265350341797


array([1, 1, 1, 0, 0, 1, 1, 0, 1, 1])

In [ ]:
loaded_model_predict("XLMR")

[0 0 1 0 0 1 1 1 1 1]
Prediction Time: 32.86075973510742


array([0, 0, 1, 0, 0, 1, 1, 1, 1, 1])

In [ ]:
loaded_model_predict("BERTPTL")

[0 0 1 0 0 1 1 0 0 1]
Prediction Time: 40.95278239250183


array([0, 0, 1, 0, 0, 1, 1, 0, 0, 1])

# MULTILABEL PREDICTION

In [ ]:
loaded_model_predict("BERTPT_ML", multilabel=True)

[[0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0.]]
Prediction Time: 9.870826482772827


array([[0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0.]], dtype=float32)

In [ ]:
loaded_model_predict("DEBERTA_ML", multilabel=True)

[[0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0.]]
Prediction Time: 18.725408792495728


array([[0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0.]], dtype=float32)

# SENTENCE SIMILARITY ASSESSMENT

In [15]:
def generate_top5(queries):
  # Loads the Sentence-BERT model
  embedder = SentenceTransformer('distiluse-base-multilingual-cased-v2')
  # Loads the dataset containing information about the similar texts
  df = pd.read_csv("/content/gdrive/MyDrive/Datasets/dataset_verifato_sentence_similarity.csv", sep=";")
  # Transforms the claimReviewed column into a list
  df_to = df['claimReviewed'].to_list()
  # Encodes the dataset
  corpus_embeddings = embedder.encode(df_to)
  # Reads the embeddings of the dataset's claimReviewed column previously encoded
  obj = pd.read_pickle(r'/content/gdrive/MyDrive/Datasets/sts-embeddings.pkl')
  top5 = []

  for query in queries:
    query_embedding = embedder.encode(query)
    intro = f"Query: {query}"
    title = f"\nTop 5 of Similar Claims: "

    # Perform semantic search and retrieve the top 5 hits
    hits = util.semantic_search(query_embedding, corpus_embeddings, top_k=5)
    hits = hits[0]

    # Iterate through the hits and print information about the similar claims
    for hit in hits:
      for i in range(len(df)):
        if df_to[hit['corpus_id']] == df['claimReviewed'][i]:
          top_iter = f"Claim Verified: {df_to[hit['corpus_id']]}, \n(Score: {(hit['score']):.4f})\n Published Date: {df['datePublished'][i]}\n Link: {df['link'][i]}\n Assessment: {df['label'][i]}\n"
          top5.append(top_iter)
  return f"{intro}\n{title}{{'\n'.join(top5)}}"

In [16]:
generate_top5(prediction_corpus)

"Query: O câncer começa quando as alterações genéticas interferem na replicação normal e na reposição das células do corpo. As células começam a crescer descontroladamente e podem formar um tumor. É a segunda principal causa de morte nos Estados Unidos. Infelizmente, parece que a doença pode estar aumentando graças às injeções experimentais de Covid-19. Porque os dados oficiais do governo dos EUA confirmam que o risco de desenvolver câncer após a vacinação contra o Covid-19 aumenta em chocantes 143.233%\n\nTop 5 of Similar Claims: {'\n'.join(top5)}"

# FINAL FUNCTION DEMONSTRATION ON SAMPLE QUERY

In [17]:
# Defining the sample query as the user would provide
sample_query = ['Um novo estudo prospectivo que inclui mais de 223.000 participantes do sul do Brasil descobriu que a ivermectina reduz a mortalidade por covid-19 em 92% . O uso de ivermectina foi associado a reduções não apenas na mortalidade por COVID-19, mas também nas hospitalizações e nas taxas de infecção. Como profilático, a ivermectina mostrou-se mais eficaz de maneira dose-dependente. À medida que a dose de ivermectina aumentava, os pacientes tinham maior probabilidade de se recuperar da covid-19. Evidências preliminares também mostram que os pacientes se recuperam mais rapidamente com ivermectina .']

In [18]:
# Final function calls the function loaded_model_Predict once for the first feature of the application
# (Is the input data a case of disinformation or not?) and then for the second feature (What are the Covid-related themes?),
# transforming the positive output into a list of indexes, to be translated to the themes in the front end.
# Next, it runs the function generate_top_5. The final output is transformed into a JSON string,
# which is the method of transfer from the backend to the front end.
# As all tokenizers and models are saved into the application server, the load time is definitely viable for a web application.
# As such, this function was run again after the loading of the tokenizer and the model to exemply a normal run.

def getOutputAsJson():
  start = time.time()

  feature_fake_or_not = loaded_model_predict("BERTPT", dataset = sample_query)

  feature_theme_output = loaded_model_predict("DEBERTA_ML", dataset = sample_query, multilabel=True)
  positives_outcomes = np.where(feature_theme_output == 1)
  feature_theme = positives_outcomes[1].tolist()

  feature_top5_similar_results = generate_top5(sample_query)
  end = time.time()
  total_time = end - start
  data = {
    'Probability' : str(feature_fake_or_not),

    'Covid Themes' : str(feature_theme),

    'Similar Claims TOP 5' : feature_top5_similar_results
  }
  print(f"Total Time: {total_time}")
  json_string = json.dumps(data,ensure_ascii=False)
  return json_string

In [20]:
json_string = getOutputAsJson()
json_string

[1]
Prediction Time: 1.5511224269866943


[[0. 0. 0. 0. 1. 0. 0. 0.]]
Prediction Time: 4.542011022567749
Total Time: 14.4563148021698


'{"Probability": "[1]", "Covid Themes": "[4]", "Similar Claims TOP 5": "Query: Um novo estudo prospectivo que inclui mais de 223.000 participantes do sul do Brasil descobriu que a ivermectina reduz a mortalidade por covid-19 em 92% . O uso de ivermectina foi associado a reduções não apenas na mortalidade por COVID-19, mas também nas hospitalizações e nas taxas de infecção. Como profilático, a ivermectina mostrou-se mais eficaz de maneira dose-dependente. À medida que a dose de ivermectina aumentava, os pacientes tinham maior probabilidade de se recuperar da covid-19. Evidências preliminares também mostram que os pacientes se recuperam mais rapidamente com ivermectina .\\n\\nTop 5 of Similar Claims: {\'\\n\'.join(top5)}"}'

In [21]:
# This is a way to save the JSON string to upload to GitHub as an example
def saveJsonFile(json_string):
  with open('json_data.json', 'w', encoding='utf-8') as outfile:
    outfile.write(json_string)

In [22]:
saveJsonFile(json_string)